In [1]:
import pandas as pd
import numpy as np

In [2]:
def assign_consistent_team_codes(df):
    ''' 
        The function assumes that the dataframe has a home_team_code column which has been assigned using pandas like above
     '''
    # Create a dictionary to store team names and their codes
    team_codes = {}

    # Assign codes for home teams
    for index, row in df.iterrows():
        if row['home_team'] not in team_codes:
            team_codes[row['home_team']] = row['home_team_code']
    
    # Assign consistent codes for both home and away teams
    df['home_team_code'] = df['home_team'].map(team_codes)
    df['away_team_code'] = df['away_team'].map(team_codes)

    # # Verify the results
    # print("Team Codes:")
    # for team, code in sorted(team_codes.items(), key=lambda x: x[1]):
    #     print(f"{team}: {code}")


    # Check for any NaN values in team codes (which would indicate a missing team)
    if df['home_team_code'].isnull().any() or df['away_team_code'].isnull().any():
        print("\nWarning: Some teams don't have assigned codes. Please check your data.")
    else:
        print("\nAll teams have been assigned consistent codes.")

    # Verify if the number of unique teams matches the number of unique codes
    num_teams = len(set(df['home_team']) | set(df['away_team']))
    num_codes = len(set(df['home_team_code']) | set(df['away_team_code']))
    
    # print(f"\nNumber of unique teams: {num_teams}")
    # print(f"Number of unique codes: {num_codes}")
    
    # if num_teams == num_codes:
        # print("Verification successful: Number of unique teams matches number of unique codes.")
    # else:
        # print("Verification failed: Number of unique teams does not match number of unique codes.")

    return df, team_codes

In [3]:
def calculate_trailing_average(series):
    result = []
    for i in range(len(series)):
        if i == 0:
            result.append(0)  # First game has no trailing average
        else:
            # Calculate average of all values up to (but not including) current index
            result.append(series.iloc[:i].mean())
    return result


In [4]:
# df = pd.DataFrame({})
years = ['2023_2024', '2022_2023', '2021_2022', '2018_2019', '2017_2018', '2016_2017', '2015_2016', '2014_2015', '2013_2014']

all_data = pd.DataFrame({})

for year in years:
    df = pd.read_json(f'../season_data/game_data_premier_league_{year}.json')
    df['season'] = year
    df['round'] = df['round'].apply(lambda x: int(x.split()[1]) if isinstance(x, str) else x)
    df['date'] = pd.to_datetime(df['date'])
    # df.info()
    df['home_team_code'] = df['home_team'].astype('category').cat.codes
    assign_consistent_team_codes(df)
    # Relevant attributes
    relevant_columns =  ['season', 'round', 'date', 'time', 'home_team', 'away_team', 'home_goals', 'away_goals',
        'home_ball_possession', 'away_ball_possession', 'home_goal_attempts', 'away_goal_attempts', 'home_shots_on_goal', 'away_shots_on_goal',
            'home_corner_kicks', 'away_corner_kicks', 'home_goalkeeper_saves', 'away_goalkeeper_saves', 'home_attacks', 'away_attacks',
        'home_dangerous_attacks', 'away_dangerous_attacks', 'home_blocked_shots', 'away_blocked_shots', 'home_team_code', 'away_team_code']
    # Next, I will only include the columns that are in the original dataframe
    present_columns = [column for column in relevant_columns if column in df.columns]
    df = df[present_columns]
    df = df.rename(columns={'home_goal_attempts': 'home_shots', 'away_goal_attempts': 'away_shots', 'home_ball_possession': 'home_poss', 'away_ball_possession': 'away_poss'})
    # Fill out missing values
    # Check if columns exist before filling missing values
    columns_to_fill = ['home_blocked_shots', 'away_blocked_shots', 'home_attacks', 'away_attacks', 'home_dangerous_attacks', 'away_dangerous_attacks']
    for column in columns_to_fill:
        if column in df.columns:
            df[column] = df[column].fillna(0)
        else:
            df[column] = 0
    # Check for missing values
    null_values = df.isnull().sum()
    df['home_shots_off_goal'] = df['home_shots'] - df['home_shots_on_goal'] - df['home_blocked_shots']
    df['away_shots_off_goal'] = df['away_shots'] - df['away_shots_on_goal'] - df['away_blocked_shots']
    df['home_chances'] = df['home_corner_kicks'] + df['home_attacks']
    df['away_chances'] = df['away_corner_kicks'] + df['away_attacks']
    # Possession stats should be normalized values between 0 - 1
    df['home_poss'] = df['home_poss'].apply(lambda x: float(x.strip('%'))/100 if isinstance(x, str) else x)
    df['away_poss'] = df['away_poss'].apply(lambda x: float(x.strip('%'))/100 if isinstance(x, str) else x)
    # Deriving special metrics:
    # Replace 0 with 1 in all denominator columns to avoid division by zero
    df['home_chances'] = df['home_chances'].replace(0, 1)
    df['away_chances'] = df['away_chances'].replace(0, 1)
    df['home_shots'] = df['home_shots'].replace(0, 1)
    df['away_shots'] = df['away_shots'].replace(0, 1)
    df['home_shots_on_goal'] = df['home_shots_on_goal'].replace(0, 1)
    df['away_shots_on_goal'] = df['away_shots_on_goal'].replace(0, 1)

    # Shot creation ratio. Tells how much shots a team has from their chances created
    df['home_shot_creation_ratio'] = df['home_shots'] / df['home_chances']
    df['away_shot_creation_ratio'] = df['away_shots'] / df['away_chances']

    # Target rate. Gives a good measure of how accurate a team's attempts are
    df['home_target_ratio'] = df['home_shots_on_goal'] / df['home_shots']
    df['away_target_ratio'] = df['away_shots_on_goal'] / df['away_shots']

    # Conversion rate. Gives a good measure of goals to shots taken
    df['home_conversion_rate'] = df['home_goals'] / df['home_shots']
    df['away_conversion_rate'] = df['away_goals'] / df['away_shots']

    # Target to goal ratio. Gives a measure of goals scored to shots on target
    df['home_target_to_goal_ratio'] = df['home_goals'] / df['home_shots_on_goal']
    df['away_target_to_goal_ratio'] = df['away_goals'] / df['away_shots_on_goal']
    # Calculate rolling averages for all relevant metrics
    # Create team-level data for rolling averages
    # First for home teams
    home_metrics = {
        'goals': 'home_goals',
        'conceded_goals': 'away_goals',
        'shots': 'home_shots',
        'conceded_shots': 'away_shots',
        'shots_on_goal': 'home_shots_on_goal',
        'conceded_shots_on_goal': 'away_shots_on_goal',
        'goalkeeper_saves': 'home_goalkeeper_saves',
        'blocked_shots': 'home_blocked_shots',
        'shots_off_goal': 'home_shots_off_goal',
        'chances': 'home_chances',
        'shot_creation_ratio': 'home_shot_creation_ratio',
        'target_ratio': 'home_target_ratio',
        'conversion_rate': 'home_conversion_rate',
        'target_to_goal_ratio': 'home_target_to_goal_ratio'
    }
    
    home_df = df[['round', 'home_team_code'] + list(home_metrics.values())].copy()
    home_df.columns = ['round', 'team'] + list(home_metrics.keys())
    
    # Then for away teams
    away_metrics = {k: v.replace('home_', 'away_') for k, v in home_metrics.items()}
    away_df = df[['round', 'away_team_code'] + list(away_metrics.values())].copy()
    away_df.columns = ['round', 'team'] + list(away_metrics.keys())
    
    # Combine and sort
    team_df = pd.concat([home_df, away_df])
    team_df = team_df.sort_values(['team', 'round'])
    
    # Calculate trailing averages
    metrics_to_average = list(home_metrics.keys())
    for metric in metrics_to_average:
        team_df[f'{metric}_rolling_avg'] = team_df.groupby('team')[metric].transform(
            calculate_trailing_average
        )
    
    # Merge trailing averages back to original dataframe
    # For home teams
    home_avgs = team_df[['round', 'team'] + [f'{m}_rolling_avg' for m in metrics_to_average]]
    home_avgs.columns = ['round', 'home_team_code'] + [f'home_{m}_rolling_avg' for m in metrics_to_average]
    df = pd.merge(df, home_avgs, on=['round', 'home_team_code'])
    
    # For away teams
    away_avgs = team_df[['round', 'team'] + [f'{m}_rolling_avg' for m in metrics_to_average]]
    away_avgs.columns = ['round', 'away_team_code'] + [f'away_{m}_rolling_avg' for m in metrics_to_average]
    df = pd.merge(df, away_avgs, on=['round', 'away_team_code'])
    
    df.sort_values(by='round', inplace=True)
    df = df.dropna()
    # print(df.head(20))
    all_data = pd.concat([all_data, df], ignore_index=True)


all_data.to_json('../processed_data/pl_data_dt.json', orient='records', lines=True)

    # df = pd.concat([df, test_df], ignore_index=True)

# print(df.shape)
# df.head()


All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.

All teams have been assigned consistent codes.


Original data:
   round team  goals
0      1    A      1
1      1    B      2
2      2    A      4
3      2    B      1
4      3    A      3
5      3    B      2

With trailing averages:
   round team  goals  goals_trailing_avg
0      1    A      1                 0.0
2      2    A      4                 1.0
4      3    A      3                 2.5
1      1    B      2                 0.0
3      2    B      1                 2.0
5      3    B      2                 1.5
